In [1]:
import os
import getpass
from typing import Annotated, Dict, List, Any
from typing_extensions import TypedDict

def setup_environment():
    """Setup your API Keys"""
    if not os.environ.get("OPENAI_API_KEY"):
        openai_key = getpass.getpass("Enter your OpenAI API Key: ")
        os.environ["OPENAI_API_KEY"] = openai_key
    if not os.environ.get("LANGSMITH_API_KEY"):
        langsmith_key = getpass.getpass("Enter your LangSmith API Key: ")
        if langsmith_key:
            os.environ["LANGSMITH_API_KEY"] = langsmith_key
            os.environ["LANGCHAIN_TRACING_V2"] = "true"
            os.environ["LANGCHAIN_PROJECT"] = "LangGraph Basic"
        else:
            print("LangSmith API Key not provided. Tracing will not be enabled.")
            
    print("Enviroment setup completed.")

setup_environment()


Enviroment setup completed.


In [2]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

class State(TypedDict):
    # messages will store our converstaion history
    # add_messages is a special function that appends new message instead of replacing them
    messages: Annotated[list, add_messages]

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model = "gpt-4o",
    temperature = 0.7
)
llm.invoke("Hi").content

'Hello! How can I assist you today?'

In [4]:
def chatbot_node(state: State) -> Dict[str, Any]:
    messages = state['messages']
    response = llm.invoke(messages)
    return {"messages": [response]}

In [5]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

simple_chatbot = graph_builder.compile()

In [6]:
initial_state = {
    "messages": [HumanMessage(content="Hello! My name is Manmath. What is your name?")]
}

result = simple_chatbot.invoke(initial_state)

In [7]:
result['messages']

[HumanMessage(content='Hello! My name is Manmath. What is your name?', additional_kwargs={}, response_metadata={}, id='66341326-910b-4836-b9d3-78b9bd8cf7ae'),
 AIMessage(content="Hello Manmath! I'm an AI language model created by OpenAI, and I don't have a personal name like a human does. You can call me Assistant. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 20, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C6JZV5xPdSB4UG1nkAqoPLjcmD16G', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0bdf2ae1-d93e-45f9-90f2-c88d604e92ab-0', usage_metadata={'input_tokens': 20, 'output_tokens': 39, 'total_to

In [8]:
for i, message in enumerate(result['messages']):
    if isinstance(message, HumanMessage):
        print(f"User: {message.content}")
    if isinstance(message, AIMessage):
        print(f"AI: {message.content}")

User: Hello! My name is Manmath. What is your name?
AI: Hello Manmath! I'm an AI language model created by OpenAI, and I don't have a personal name like a human does. You can call me Assistant. How can I assist you today?


In [9]:
def test_simple_chatbot(user_input):
    initial_state = {
        "messages": [HumanMessage(content=user_input)]
    }
    result = simple_chatbot.invoke(initial_state)
    for i, message in enumerate(result['messages']):
        if isinstance(message, HumanMessage):
            print(f"User: {message.content}")
        if isinstance(message, AIMessage):
            print(f"AI: {message.content}")
    return result

In [10]:
test_simple_chatbot("Who was the war secretary of the US under Abraham Lincoln?")

User: Who was the war secretary of the US under Abraham Lincoln?
AI: The Secretary of War under President Abraham Lincoln was Edwin M. Stanton. He served from 1862 to 1868, following Simon Cameron, who was the Secretary of War at the beginning of Lincoln's presidency. Stanton played a crucial role during the Civil War, managing the Union Army's logistics and operations.


{'messages': [HumanMessage(content='Who was the war secretary of the US under Abraham Lincoln?', additional_kwargs={}, response_metadata={}, id='666598d7-03e3-48fc-ae49-ed38f0fa0408'),
  AIMessage(content="The Secretary of War under President Abraham Lincoln was Edwin M. Stanton. He served from 1862 to 1868, following Simon Cameron, who was the Secretary of War at the beginning of Lincoln's presidency. Stanton played a crucial role during the Civil War, managing the Union Army's logistics and operations.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 19, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_80956533cb', 'id': 'chatcmpl-C6JZXVW2pH3P3TsGitzdLpf4vq24M', 'service_tier': 'defau

In [11]:
test_simple_chatbot("What was my last question?")

User: What was my last question?
AI: I'm sorry, but I don't have the ability to recall previous interactions or questions. Each session is independent and private. If you have a question, feel free to ask again!


{'messages': [HumanMessage(content='What was my last question?', additional_kwargs={}, response_metadata={}, id='d67f8cf5-8c0d-4026-8956-2f4e37abbb79'),
  AIMessage(content="I'm sorry, but I don't have the ability to recall previous interactions or questions. Each session is independent and private. If you have a question, feel free to ask again!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 13, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ea40d5097a', 'id': 'chatcmpl-C6JZZcoL1rfskQN4tGC3NQKipr9Cv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--349a88fd-fd64-4cc0-87b5-4b809cb2c316-0', usage_metadata={'input_tokens': 13, 'output_tokens': 35, 'tota

In [13]:
%pip install langgraph-checkpoint-sqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langgraph-checkpoint-sqlite]
Note: you may need to restart the kernel to use updated packages.


In [20]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

### CREATING CHATBOT WITH MEMORY

In [22]:
graph_builder_with_memory = StateGraph(State)
graph_builder_with_memory.add_node("chatbot", chatbot_node)
graph_builder_with_memory.add_edge(START, "chatbot")
graph_builder_with_memory.add_edge("chatbot", END)

chatbot_with_memory = graph_builder_with_memory.compile(checkpointer=memory)


In [23]:
import uuid
unique_id = uuid.uuid4()
print(f"Unique ID: {unique_id}")

Unique ID: dfb86aec-ae38-4ce6-a23f-86fc9a539a71


In [24]:
def test_simple_chatbot_with_memory(user_input):
    config = {"configurable": {"thread_id": str(unique_id)}}
    initial_state = {
        "messages": [HumanMessage(content=user_input)]
    }
    result = chatbot_with_memory.invoke(initial_state, config)
    return result

In [26]:
test_simple_chatbot_with_memory("Hi My name is Manmath, I aspire to be an AI Engineer")

{'messages': [HumanMessage(content='Hi My name is Manmath, I aspire to be an AI Engineer', additional_kwargs={}, response_metadata={}, id='4e5ee263-d5cc-432d-9205-670106860a24'),
  AIMessage(content="Hi Manmath! That's a great aspiration. AI engineering is a dynamic and exciting field with a lot of potential for innovation and impact. Here are a few steps you can consider on your journey to becoming an AI Engineer:\n\n1. **Educational Foundation**: \n   - **Mathematics and Statistics**: Make sure you have a strong grasp of linear algebra, calculus, probability, and statistics, as these are crucial for understanding AI algorithms.\n   - **Computer Science**: Gain proficiency in data structures, algorithms, and software development. Familiarity with programming languages such as Python, Java, or C++ is essential.\n\n2. **Learn AI Concepts**:\n   - **Machine Learning**: Start with the basics of supervised and unsupervised learning, and then delve into more advanced topics like deep learni

In [27]:
test_simple_chatbot_with_memory("What is my name?")

{'messages': [HumanMessage(content='Hi My name is Manmath, I aspire to be an AI Engineer', additional_kwargs={}, response_metadata={}, id='4e5ee263-d5cc-432d-9205-670106860a24'),
  AIMessage(content="Hi Manmath! That's a great aspiration. AI engineering is a dynamic and exciting field with a lot of potential for innovation and impact. Here are a few steps you can consider on your journey to becoming an AI Engineer:\n\n1. **Educational Foundation**: \n   - **Mathematics and Statistics**: Make sure you have a strong grasp of linear algebra, calculus, probability, and statistics, as these are crucial for understanding AI algorithms.\n   - **Computer Science**: Gain proficiency in data structures, algorithms, and software development. Familiarity with programming languages such as Python, Java, or C++ is essential.\n\n2. **Learn AI Concepts**:\n   - **Machine Learning**: Start with the basics of supervised and unsupervised learning, and then delve into more advanced topics like deep learni

In [29]:
test_simple_chatbot_with_memory("What was my last question?")

{'messages': [HumanMessage(content='Hi My name is Manmath, I aspire to be an AI Engineer', additional_kwargs={}, response_metadata={}, id='4e5ee263-d5cc-432d-9205-670106860a24'),
  AIMessage(content="Hi Manmath! That's a great aspiration. AI engineering is a dynamic and exciting field with a lot of potential for innovation and impact. Here are a few steps you can consider on your journey to becoming an AI Engineer:\n\n1. **Educational Foundation**: \n   - **Mathematics and Statistics**: Make sure you have a strong grasp of linear algebra, calculus, probability, and statistics, as these are crucial for understanding AI algorithms.\n   - **Computer Science**: Gain proficiency in data structures, algorithms, and software development. Familiarity with programming languages such as Python, Java, or C++ is essential.\n\n2. **Learn AI Concepts**:\n   - **Machine Learning**: Start with the basics of supervised and unsupervised learning, and then delve into more advanced topics like deep learni

### TOOL INTEGRATION

Real AI agents need to interact with the outside world! Let's add the tool calling capabilities

### WHAT ARE TOOLS 

- search the web
- Query Database
- send email
- perform calculations
- Access APIs

In [30]:
import math

In [31]:
from langchain_core.tools import tool

@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression safely.
    
    Args:
        expression: A mathematical expression to evaluate (e.g., "2 + 3 * 4")
        
    Returns:
        The result of the calculation
    """
    try:
        # Safe evaluation of mathematical expressions
        # Only allow basic math operations
        allowed_names = {
            k: v for k, v in math.__dict__.items() if not k.startswith("__")
        }
        allowed_names.update({"abs": abs, "round": round})
        
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"


In [34]:
calculator("sqrt(16)")

'The result of sqrt(16) is 4.0'